In [6]:
import numpy as np
from exputils.dot.dot_product import compute_all_dot_products
from exputils.RoM.dot import calculate_RoM_dot
from exputils.state.random import make_random_quantum_state

In [7]:
def lb_by_dot(n_qubit, rho_vec):
    dots = compute_all_dot_products(n_qubit, rho_vec)
    dp = np.max(dots)
    dm = np.min(dots)
    assert np.isclose(rho_vec.T @ rho_vec, np.linalg.norm(rho_vec, ord=2) ** 2)
    return 1 + 2 * max(0, (np.linalg.norm(rho_vec, ord=2) ** 2 - dp) / (dp - dm))


def lb_by_stabilizer_fidelity(n_qubit, rho_vec):
    # valid only when the state is pure
    dots = compute_all_dot_products(n_qubit, rho_vec)
    stabilizer_fidelity = np.max(dots) / (2**n_qubit)
    return 1 / stabilizer_fidelity

In "Simulation of quantum circuits by low-rank stabilizer decompositions"

(Sergey Bravyi, Dan Browne, Padraic Calpin, Earl Campbell, David Gosset, and Mark Howard)

It is said that $F(T) \approx 0.853$.

We confirm this.


In [8]:
from exputils.state.tensor import make_random_tensor_product_state

n_qubit = 1
rho_vec = make_random_tensor_product_state("H", n_qubit, seed=0)
dots = compute_all_dot_products(n_qubit, rho_vec)
stabilizer_fidelity = np.max(dots) / (2**n_qubit)
print(stabilizer_fidelity)

0.8535533905932737


In [9]:
n_qubit = 4

for kind in ["pure", "mixed"]:
    print("=" * 10, f"{kind=}", "=" * 10)
    for seed in range(10):
        rho_vec = make_random_quantum_state(kind, n_qubit, seed=seed)
        RoM, coeff, Amat = calculate_RoM_dot(n_qubit, rho_vec, method="gurobi")
        lb_sf = lb_by_stabilizer_fidelity(n_qubit, rho_vec)
        print(f"{lb_sf=} {RoM=} {lb_sf <= RoM=}")
        assert lb_sf <= RoM

========== kind='pure' ==========
lb_sf=1.7848266417977803 RoM=4.331854824887728 lb_sf <= RoM=True
lb_sf=2.0879616272346215 RoM=4.302359569370483 lb_sf <= RoM=True
lb_sf=1.9674600518646173 RoM=4.381970215694802 lb_sf <= RoM=True
lb_sf=1.6791792186872825 RoM=4.230197583759634 lb_sf <= RoM=True
lb_sf=1.6942965885644286 RoM=4.22947512794538 lb_sf <= RoM=True
lb_sf=1.9150265864033158 RoM=4.332809129965167 lb_sf <= RoM=True
lb_sf=2.1359366876427126 RoM=4.517429721127863 lb_sf <= RoM=True
lb_sf=1.72285500107303 RoM=4.244393100644842 lb_sf <= RoM=True
lb_sf=1.8867955901216782 RoM=4.301049017374336 lb_sf <= RoM=True
lb_sf=2.0601721143546285 RoM=4.495309081053528 lb_sf <= RoM=True
========== kind='mixed' ==========
lb_sf=8.090671610821515 RoM=1.37967595211064 lb_sf <= RoM=False


AssertionError: 